In [61]:
#show all output in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import packages
import pandas as pd
import numpy as np
#read in dataset
df = pd.read_csv('hbcu.csv')
#print out column names
print(df.columns)

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID'],
      dtype='object')


In [62]:
#get data type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    102 non-null    object 
 1   Address                 105 non-null    object 
 2   Website                 102 non-null    object 
 3   Type                    102 non-null    object 
 4   Awards offered          102 non-null    object 
 5   Campus setting          101 non-null    object 
 6   Campus housing          102 non-null    object 
 7   Student population      100 non-null    float64
 8   Undergraduate students  97 non-null     float64
 9   Graduation Rate         96 non-null     object 
 10  Transfer-Out Rate       63 non-null     object 
 11  Cohort Year *           102 non-null    object 
 12  Net Price **            96 non-null     object 
 13  Largest Program         102 non-null    object 
 14  IPEDS ID                102 non-null    fl

In [63]:
#clean dataset and create variables
#create new highest degree column
#get unique values for 'awards offered' column
df['Awards offered'].unique()

array(["Bachelor's degree|Master's degree|Post-master's certificate|Doctor's degree - research/scholarship",
       "Bachelor's degree|Postbaccalaureate certificate|Master's degree|Post-master's certificate|Doctor's degree - research/scholarship|Doctor's degree - professional practice",
       "Less than one year certificate|One but less than two years certificate|Associate's degree|Bachelor's degree|Master's degree|Post-master's certificate",
       "Associate's degree|Bachelor's degree|Master's degree|Post-master's certificate|Doctor's degree - professional practice",
       "Associate's degree|Bachelor's degree|Master's degree|Doctor's degree - research/scholarship",
       "Associate's degree|Bachelor's degree",
       "Bachelor's degree|Master's degree", "Bachelor's degree",
       "Bachelor's degree|Postbaccalaureate certificate|Master's degree",
       "Less than one year certificate|One but less than two years certificate|Associate's degree",
       "Less than one year certific

In [64]:
#find and fill in NaN values
df['Awards offered'].isna().sum()
df['Awards offered'] = df['Awards offered'].fillna('Missing')
df['Awards offered'].isna().sum()

3

0

In [65]:
df['Doctor'] = df['Awards offered'].str.contains('Doctor')
df['Master'] = df['Awards offered'].str.contains('Master')
df['Bachelor'] = df['Awards offered'].str.contains('Bachelor')
df['Associate'] = df['Awards offered'].str.contains('Associate')

In [95]:
#create highest degree column
df['Highest Degree'] = 'Unknown'
df.loc[df['Associate'] == True, 'Highest Degree'] = 'Associate' 
df.loc[df['Bachelor'] == True, 'Highest Degree'] = 'Bachelor' 
df.loc[df['Master'] == True, 'Highest Degree'] = 'Master' 
df.loc[df['Doctor'] == True, 'Highest Degree'] = 'Doctor' 
df['Highest Degree'].value_counts()


Highest Degree
Doctor       40
Master       33
Bachelor     18
Associate    11
Unknown       3
Name: count, dtype: int64